## 소방서 data

In [1]:
import requests
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
firehouse = pd.read_csv(
    filepath_or_buffer = "../data/소방서 데이터.csv", 
    encoding = "cp949"
    )
firehouse = firehouse[firehouse["본부명"] == "경남소방본부"].reset_index(drop=True)

,순번,본부명,소방서,주소,전화번호,FAX
0,200,경남소방본부,거제소방서,경상남도 거제시 진목로 1,055-689-9212,055-689-9219
1,201,경남소방본부,거창소방서,경상남도 거창군 거창읍 거함대로 3324,055-940-9212,055-940-9219
2,202,경남소방본부,고성소방서,경상남도 고성군 고성읍 남해안대로 2670,055-670-9212,055-670-9219
3,203,경남소방본부,김해동부소방서,경상남도 김해시 김해대로 2507,055-320-9212,055-320-9219
4,204,경남소방본부,김해서부소방서,경상남도 김해시 장유로 341,055-344-9212,055-344-9219
5,205,경남소방본부,남해소방서,경상남도 남해군 남해읍 스포츠로 81,055-860-9212,055-860-9219
6,206,경남소방본부,밀양소방서,경상남도 밀양시 밀양대로 1760,055-350-9212,055-350-9219
7,207,경남소방본부,사천소방서,경상남도 사천시 남일로 29,055-830-9212,055-830-9219
8,208,경남소방본부,산청소방서,경상남도 산청군 산청읍 중앙로 76-7,055-970-9212,055-970-9219
9,209,경남소방본부,양산소방서,경상남도 양산시 물금읍 황산로 719,055-379-9212,055-379-9219


In [3]:
# API 엔드포인트
apiurl = "https://api.vworld.kr/req/address?"

# API 키
api_key = "8B7A9BDC-2FB8-3A2E-8F1C-359C7D1882B8"

# # 주소가 포함된 데이터프레임 예시
# data = {
#     "주소": ["서울특별시 강남구 테헤란로 123", "부산광역시 해운대구 우동 123"]
# }

# df = pd.DataFrame(data)

# 좌표 정보를 저장할 빈 리스트 생성
coordinates = []


# 데이터프레임의 각 행을 순회하며 주소를 좌표로 변환
for index, row in firehouse.iterrows():
    address = row['주소']
    params = {
        "service": "address",
        "request": "getcoord",
        "crs": "epsg:4326",
        "address": address,
        "format": "json",
        "type": "road",
        "key": api_key
    }
    response = requests.get(apiurl, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if data['response']['status'] == 'OK':
            x = data['response']['result']['point']['x']
            y = data['response']['result']['point']['y']
            coordinates.append({"경도": x, "위도": y})
        else:
            coordinates.append({"경도": None, "위도": None})
    else:
        coordinates.append({"경도": None, "위도": None})

# 결과를 데이터프레임으로 변환
coord_df = pd.DataFrame(coordinates)


,경도,위도
0,128.686162892,34.896160085
1,127.923447442,35.677620484
2,128.331066159,34.981093394
3,128.901161541,35.228565412
4,128.817395559,35.196596896
5,127.892610368,34.829581711
6,128.753405357,35.482811678
7,128.079914824,34.933220970
8,127.883835023,35.415077468
9,129.010028030,35.334037335


In [4]:
# 데이터 합치기
firehouse = pd.concat([firehouse, coord_df], axis=1)

,순번,본부명,소방서,주소,전화번호,FAX,경도,위도
0,200,경남소방본부,거제소방서,경상남도 거제시 진목로 1,055-689-9212,055-689-9219,128.686162892,34.896160085
1,201,경남소방본부,거창소방서,경상남도 거창군 거창읍 거함대로 3324,055-940-9212,055-940-9219,127.923447442,35.677620484
2,202,경남소방본부,고성소방서,경상남도 고성군 고성읍 남해안대로 2670,055-670-9212,055-670-9219,128.331066159,34.981093394
3,203,경남소방본부,김해동부소방서,경상남도 김해시 김해대로 2507,055-320-9212,055-320-9219,128.901161541,35.228565412
4,204,경남소방본부,김해서부소방서,경상남도 김해시 장유로 341,055-344-9212,055-344-9219,128.817395559,35.196596896
5,205,경남소방본부,남해소방서,경상남도 남해군 남해읍 스포츠로 81,055-860-9212,055-860-9219,127.892610368,34.829581711
6,206,경남소방본부,밀양소방서,경상남도 밀양시 밀양대로 1760,055-350-9212,055-350-9219,128.753405357,35.482811678
7,207,경남소방본부,사천소방서,경상남도 사천시 남일로 29,055-830-9212,055-830-9219,128.079914824,34.933220970
8,208,경남소방본부,산청소방서,경상남도 산청군 산청읍 중앙로 76-7,055-970-9212,055-970-9219,127.883835023,35.415077468
9,209,경남소방본부,양산소방서,경상남도 양산시 물금읍 황산로 719,055-379-9212,055-379-9219,129.010028030,35.334037335


In [5]:
firehouse = firehouse[['소방서','주소','경도','위도']]
firehouse.rename(columns={'소방서':'시설명'}, inplace=True)

,시설명,주소,경도,위도
0,거제소방서,경상남도 거제시 진목로 1,128.686162892,34.896160085
1,거창소방서,경상남도 거창군 거창읍 거함대로 3324,127.923447442,35.677620484
2,고성소방서,경상남도 고성군 고성읍 남해안대로 2670,128.331066159,34.981093394
3,김해동부소방서,경상남도 김해시 김해대로 2507,128.901161541,35.228565412
4,김해서부소방서,경상남도 김해시 장유로 341,128.817395559,35.196596896
5,남해소방서,경상남도 남해군 남해읍 스포츠로 81,127.892610368,34.829581711
6,밀양소방서,경상남도 밀양시 밀양대로 1760,128.753405357,35.482811678
7,사천소방서,경상남도 사천시 남일로 29,128.079914824,34.933220970
8,산청소방서,경상남도 산청군 산청읍 중앙로 76-7,127.883835023,35.415077468
9,양산소방서,경상남도 양산시 물금읍 황산로 719,129.010028030,35.334037335


## 경찰서 데이터

In [15]:
police = pd.read_csv(
    filepath_or_buffer = "../data/경상남도 경찰서 데이터.csv", 
    encoding = "cp949"
    )
police.rename(columns={'소재지':'주소', '기관명':'시설명'}, inplace=True)

In [9]:
# API 엔드포인트
apiurl = "https://api.vworld.kr/req/address?"

# API 키
api_key = "8B7A9BDC-2FB8-3A2E-8F1C-359C7D1882B8"

# # 주소가 포함된 데이터프레임 예시
# data = {
#     "주소": ["서울특별시 강남구 테헤란로 123", "부산광역시 해운대구 우동 123"]
# }

# df = pd.DataFrame(data)

# 좌표 정보를 저장할 빈 리스트 생성
coordinates = []


# 데이터프레임의 각 행을 순회하며 주소를 좌표로 변환
for index, row in police.iterrows():
    address = row['주소']
    params = {
        "service": "address",
        "request": "getcoord",
        "crs": "epsg:4326",
        "address": address,
        "format": "json",
        "type": "road",
        "key": api_key
    }
    response = requests.get(apiurl, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if data['response']['status'] == 'OK':
            x = data['response']['result']['point']['x']
            y = data['response']['result']['point']['y']
            coordinates.append({"경도": x, "위도": y})
        else:
            coordinates.append({"경도": None, "위도": None})
    else:
        coordinates.append({"경도": None, "위도": None})

# 결과를 데이터프레임으로 변환
coord_df = pd.DataFrame(coordinates)

In [17]:
# 데이터 합치기
police = pd.concat([police, coord_df], axis=1)

In [18]:
police = police[['시설명','주소','경도','위도']]

In [20]:
public = pd.concat([firehouse, police],axis=0).reset_index(drop=[True])

In [22]:
public.to_csv("../data/공공시설.csv", index=False)